In [7]:
import subprocess
import time

def esegui_programma_c(n, percorso_programma,args,mpi):
    
    if mpi:
        comando = ['mpiexec','-n','4',percorso_programma] + args
    else:
        comando = [percorso_programma] + args

    for i in range(n):
        # Esegui il programma C
        risultato = subprocess.run(comando, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

n = 5 # Numero di esecuzioni

args = ['-t', '1000', '-S', 'random', '-n', '2000']  

print("Eseguendo i test con 2000 corpi")

with open("mpi-bh-times.csv", "w") as f:
    pass

with open("mpi-parallel-times.csv", "w") as f:
    pass

with open("pthread-bh-times.csv", "w") as f:
    pass

with open("pthread-parallel-times.csv", "w") as f:
    pass

with open("serial-bh-times.csv", "w") as f:
    pass
with open("serial-nbodies-times.csv", "w") as f:
    pass







print("Eseguendo i test serial_nbodies\n")
percorso_programma = '../serial_nbodies/main' 
esegui_programma_c(n, percorso_programma,args,False)

print("Eseguendo i test serial-bh\n")
percorso_programma = '../BarnesHut_serial/main'
esegui_programma_c(n,percorso_programma,args,False)

print("Eseguendo i test mpi-nbodies\n")
percorso_programma = '../mpi_parallel_nbodies/main'
esegui_programma_c(n,percorso_programma,args,True)

print("Eseguendo i test mpi-bh\n")
percorso_programma = '../mpi_BarnesHut/main'
esegui_programma_c(n,percorso_programma,args,True)

print("Eseguendo i test pthread-nbodies\n")
percorso_programma = '../pthread_parallel_nbodies/main'
esegui_programma_c(n,percorso_programma,args,False)

print("Eseguendo i test pthread-bh\n")
percorso_programma = '../pthread_BarnesHut/main'
esegui_programma_c(n,percorso_programma,args,False)

# args = ['-t', '1000', '-S', 'random', '-n', '2000']  

# print("eseguendo i test con 3000 corpi\n")

# print("Eseguendo i test serial-nbodies\n")
# percorso_programma = '../serial_nbodies/main' 
# esegui_programma_c(n, percorso_programma,args,False)

# print("Eseguendo i test serial-bh\n")
# percorso_programma = '../BarnesHut_serial/main'
# esegui_programma_c(n,percorso_programma,args,False)

# print("Eseguendo i test mpi-nbodies\n")
# percorso_programma = '../mpi_parallel_nbodies/main'
# esegui_programma_c(n,percorso_programma,args,True)

# print("Eseguendo i test mpi-bh\n")
# percorso_programma = '../mpi_BarnesHut/main'
# esegui_programma_c(n,percorso_programma,args,True)

# print("Eseguendo i test pthread-nbodies\n")
# percorso_programma = '../pthread_parallel_nbodies/main'
# esegui_programma_c(n,percorso_programma,args,False)

# print("Eseguendo i test pthread-bh\n")
# percorso_programma = '../pthread_BarnesHut/main'
# esegui_programma_c(n,percorso_programma,args,False)





Eseguendo i test con 2000 corpi
Eseguendo i test serial_nbodies

Eseguendo i test serial-bh

Eseguendo i test mpi-nbodies

Eseguendo i test mpi-bh

Eseguendo i test pthread-nbodies

Eseguendo i test pthread-bh



KeyboardInterrupt: 